In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#table版Python3，網站改版
import urllib
import gzip
import json
import requests
import mysql.connector
import datetime
import threading, time
import numpy as np #python陣列的套件，命名為np
from datetime import datetime
from threading import Thread
from urllib.request import urlretrieve

def func():
    global tStart #開始爬蟲時間
    global tEnd   #結束爬蟲時間
    global timenow#當前時間
    
    #url = "http://data.taipei/youbike"#原版Youbike網址
    url = "https://tcgbusfs.blob.core.windows.net/blobyoubike/YouBikeTP.gz"#新版Youbike網址
    urlretrieve(url, "data.gz")
    f = gzip.open('data.gz', 'r')#將gz檔解壓縮後讀取出來
    jdata = f.read()
    f.close()
    data = json.loads(jdata)#將json格式轉換成字典格式
    
    timenow = time.localtime(time.time())
    print(timenow)
    a=np.array(time.localtime(time.time()))#將時間格式轉換成陣列
    date = datetime.now().strftime('%Y_%m_%d')#取出當前時間 年_月_日
    
    if a[1]==10:#當時間來到10月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="bike_10", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    elif a[1]==11:#當時間來到11月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="bike_11", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    elif a[1]==12:#當時間來到12月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="bike_12", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    else:
        print("db not found")
        pass
    
    if True:
        print("1")
        sql="CREATE TABLE IF NOT EXISTS " + date + "(`index` bigint(20) NOT NULL AUTO_INCREMENT,`sno` int(4) NOT NULL,`tot` int(3) NOT NULL,`sbi` int(3) NOT NULL,`bemp` int(3) NOT NULL,`act` int(1) NOT NULL,`utime` datetime NOT NULL,PRIMARY KEY (`index`),KEY `sno` (`sno`)) ENGINE=InnoDB DEFAULT CHARSET=latin1 AUTO_INCREMENT=0;"
        #print(sql) #date以日期名稱命名table
        try:
            c.execute(sql) #創建table
        except Exception as e:
            print("Mysql Error %d: %s" % (e.args[0], e.args[1]))
            pass
        
        for key,value in data["retVal"].items():#用key和value將字典內的值存入變數
            sno = value["sno"]
            sna = value["sna"]
            tot = value["tot"]
            sbi = value["sbi"]
            sarea = value["sarea"]
            mday = value["mday"]
            lat = value["lat"]
            lng = value["lng"]
            ar = value["ar"]
            sareaen = value["sareaen"]
            snaen = value["snaen"]
            aren = value["aren"]
            bemp = value["bemp"]
            act = value["act"]
                
            sql = "INSERT INTO " + date + " (sno,tot,sbi,bemp,act,utime) VALUES(%s,%s,%s,%s,%s,%s)"
            try:
                c.execute(sql,(sno,tot,sbi,bemp,act,datetime.now())) #資料寫入table
            except Exception as e:
                print("Mysql Error %d: %s" % (e.args[0], e.args[1]))
                pass
        conn.commit()
        conn.close()

def start():
    t = Thread(target=func) #跑func這個function
    t.start()
   
if __name__ == '__main__':
    counter=0
    a=5
    tStart1 = time.time()
    start() #跑start這個function
    tEnd1 = time.time()
    time.sleep(20-(tEnd1 - tStart1)) #設定每20秒起來檢查一次
    tEnd1 = time.time()
    tstart0=tStart1
    
    while(a>0):
        tStart1 = tEnd1
        
        if((tStart1-tstart0)>=60): 
            counter=counter+1
        else:
            tstart0=tEnd1
            counter=counter+1
            
        if(counter%3==0): #如果到60秒起來爬資料(原始資料更新頻率)
            start()
        else:
            print("")    
            
        tEnd1 = time.time()
        time.sleep(20-(tEnd1 - tStart1))        
        tEnd1 = time.time()